### Teacher Properties

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ['LANGCHAIN_API_KEY']=os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_TRACING_V2']="true"
os.environ['LANGCHAIN_PROJECT']=os.getenv('LANGCHAIN_PROJECT')
os.environ['HF_TOKEN']=os.getenv('HF_TOKEN')
os.environ['PINECONE_API_KEY']=os.getenv('PINECONE_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

In [5]:
from langchain_groq import ChatGroq

llm = ChatGroq(groq_api_key=groq_api_key,model="Llama3-8b-8192")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001A236F4A190>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001A236878B10>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [6]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2")

d:\SL\End to End Projects\AI-Teacher\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_pinecone import PineconeVectorStore


USER_AGENT environment variable not set, consider setting it to identify your requests.
d:\SL\End to End Projects\AI-Teacher\env\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


### PDF Data

In [23]:
from langchain_community.document_loaders import PyPDFLoader
path = r'..\data\1706.03762v7.pdf'
loader = PyPDFLoader(file_path=path)

data = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=256, chunk_overlap=50)
texts = text_splitter.split_documents(data)

In [5]:
index_name = "edu-portal"


In [6]:
from pinecone import Pinecone, ServerlessSpec
import time
pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)


In [26]:
index_name = "edu-portal"

docsearch = PineconeVectorStore.from_documents(texts, embeddings, index_name=index_name)

In [27]:
query = "What is transformers ?"

retriever = docsearch.as_retriever(search_type="mmr")
matched_docs = retriever.invoke(query)
for i, d in enumerate(matched_docs):
    print(f"\n## Document {i}\n")
    print(d.page_content)


## Document 0

Figure 1: The Transformer - model architecture.
The Transformer follows this overall architecture using stacked self-attention and point-wise, fully
connected layers for both the encoder and decoder, shown in the left and right halves of Figure 1,

## Document 1

Table 2: The Transformer achieves better BLEU scores than previous state-of-the-art models on the
English-to-German and English-to-French newstest2014 tests at a fraction of the training cost.
Model
BLEU Training Cost (FLOPs)
EN-DE EN-FR EN-DE EN-FR

## Document 2

Huang & Harper (2009) [14] semi-supervised 91.3
McClosky et al. (2006) [26] semi-supervised 92.1
Vinyals & Kaiser el al. (2014) [37] semi-supervised 92.1
Transformer (4 layers) semi-supervised 92.7
Luong et al. (2015) [23] multi-task 93.0

## Document 3

in the distance between positions, linearly for ConvS2S and logarithmically for ByteNet. This makes
it more difficult to learn dependencies between distant positions [ 12]. In the Transformer this is

### Web data

In [ ]:
import bs4
loader = WebBaseLoader(
    web_paths=("https://www.infobip.com/blog/large-language-models-vs-generative-ai",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("wrapper__inner","wrapper","post-header")
        )
    )
)
docs=loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=256, chunk_overlap=50)
texts = text_splitter.split_documents(docs)

In [ ]:

docsearch = PineconeVectorStore.from_documents(texts, embeddings, index_name=index_name)


In [126]:

system_prompt = (
    "you are an professional teacher for question anwering questions"
    "use the following pieces of retrived context to answer"
    "the question. If you don't know the answer, say that you"
    "don't know, use three sentences maximum and keep the"
    "answer concise"
    "\n\n"
    "{context}"
)
propmt = ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}")
    ]
)


In [ ]:
question_answer_chain = create_stuff_documents_chain(llm,propmt)
rag_chain = create_retrieval_chain(retriever,question_answer_chain)

In [ ]:
response = rag_chain.invoke({"input":"what is transformers ?"})
print(response['answer'])

### Message History

In [128]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "Given a chat history and latest user question"
    "which might reference context in the chat history"
    "formulate a standalone question which can be understood"
    "without chat history. Do NOT answer the question"
    "just reformulate it if needed and otherwise return as it is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human","{input}")
    ]
)


In [130]:
history_aware_retrier = create_history_aware_retriever(llm,retriever,contextualize_q_prompt)
history_aware_retrier

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['PineconeVectorStore', 'HuggingFaceEmbeddings'], vectorstore=<langchain_pinecone.vectorstores.PineconeVectorStore object at 0x000001A2121D4550>, search_kwargs={}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMes

In [131]:
qa_propmt = ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human","{input}")
    ]
)

In [132]:
question_answer_chain = create_stuff_documents_chain(llm,qa_propmt)
rag_chain = create_retrieval_chain(history_aware_retrier,question_answer_chain)

In [133]:

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
        
    return store[session_id]


conversation_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer"
)

In [134]:
config = {"configurable":{"session_id":"chat_4"}}
conversation_rag_chain.invoke(
    {"input":"explain briefly ?"},
    config=config
)['answer']

'Self-attention is a mechanism that allows a model to focus on different parts of a single sequence (e.g. a sentence) to compute a representation of the sequence. It does this by comparing the sequence to itself, rather than relying solely on the previous state or input. This allows the model to weigh the importance of different parts of the sequence and combine them in a meaningful way.'

### Manage History

In [192]:
from langchain_core.messages import SystemMessage,trim_messages

trimmer_messages = trim_messages(
    max_tokens=500,
    strategy="last",
    token_counter=llm,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

In [139]:
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain_core.messages import HumanMessage

chain=(
    RunnablePassthrough.assign(messages=itemgetter("input")|trimmer_messages)|rag_chain
)

In [140]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer"
    )

In [141]:
config = {"configurable":{"session_id":"chat_4"}}
with_message_history.invoke(
    {"input":"what is attention mechanism ?"},
    config=config
)['answer']

'The attention mechanism is a method used in deep learning models to focus on specific parts of the input data that are relevant to a particular task or output. It allows the model to selectively weigh the importance of different elements in the input sequence, such as words in a sentence or notes in a melody, and combine them in a meaningful way.'

### With Agent\tool

In [1]:
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper

In [2]:
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)

In [3]:
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv=ArxivQueryRun(api_wrapper=api_wrapper_arxiv)

In [89]:
from pinecone import Pinecone, ServerlessSpec
import time
pinecone_api_key = os.environ.get("PINECONE_API_KEY")

index_name = "edu-portal"
pc = Pinecone(api_key=pinecone_api_key)

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)


In [129]:
vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)

retriever=vectorstore.as_retriever()

In [91]:
from langchain.tools.retriever import create_retriever_tool
retriver_tool = create_retriever_tool(retriever,"AI-learning","Search and return information")
retriver_tool.name

'AI-learning'

In [202]:
tools = [retriver_tool]

In [203]:

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "you are an professional teacher for question anwering questions"
            "You have access to the following tools:"
            "use the following pieces of retrived context to answer"
            "the question. If you don't know the answer, say that you"
            "don't know, use three sentences maximum and keep the"
            "answer concise"
            "\n\n"

        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)



In [209]:

from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm,tools,prompt)


In [210]:
from langchain.agents import AgentExecutor
agent_executer = AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executer

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChu

In [125]:
agent_executer.invoke({"input":"tell me about attention mechanism?"})



> Entering new AgentExecutor chain...

Invoking: `AI-learning` with `{'query': 'attention mechanism'}`


from our models and present and discuss examples in the appendix. Not only do individual attention
heads clearly learn to perform different tasks, many appear to exhibit behavior related to the syntactic
and semantic structure of the sentences.
5 Training

.
<EOS>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
Figure 3: An example of the attention mechanism following long-distance dependencies in the
encoder self-attention in layer 5 of 6. Many of the attention heads attend to a distant dependency of

<pad>
The
Law
will
never
be
perfect
,
but
its
application
should
be
just
-
this
is
what
we
are
missing
,
in
my
opinion
.
<EOS>
<pad>
Figure 5: Many of the attention heads exhibit behaviour that seems related to the structure of the

dk = dv = dmodel/h = 64. Due to the reduced dimension of each head, the total computational cost
is similar to that of single-head attention with full dimensionalit

{'input': 'tell me about attention mechanism?',
 'output': 'The attention mechanism in deep learning is a method used to help neural networks focus on specific parts of an input sequence when processing it. This is particularly useful in natural language processing tasks, such as machine translation or text summarization.'}